<a href="https://colab.research.google.com/github/michaelknorr/CHE4061/blob/main/CHE4061_GT3_Flash_Drum_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Inputs
phi_0 = float(input("Input an initial guess for phi: ")) #phi is the fraction of vapor in the feed
num_components = int(input("Input the number of streams feeding the flash drum: "))
comp_x = np.array(list(map(float, input("Input a list of molar flow rates for each stream (comma-separated): ").split(','))))
K = np.array(list(map(float, input("Input the K (equilibrium constant) values for each stream in order (comma-separated): ").split(','))))


Input an initial guess for phi: 0.3
Input the number of streams feeding the flash drum: 2
Input a list of molar flow rates for each stream (comma-separated): 30,70
Input the K (equilibrium constant) values for each stream in order (comma-separated): 3.2,0.47


In [3]:
import numpy as np

def Newton_Raphson(phi_0, num_components, comp_x, K, tol=1e-6, max_iter=100, show_table=True):
    """
    Performs Newton-Raphson iteration to find the root of a function.

    Parameters:
    phi_0 : float
        Initial guess for phi.
    num_components : int
        Number of components.
    comp_x : list or numpy array
        Mole fractions of components.
    K : list or numpy array
        Equilibrium constants of components.
    tol : float, optional
        Convergence tolerance (default is 1e-6).
    max_iter : int, optional
        Maximum number of iterations (default is 100).
    show_table : bool, optional
        Whether to print the iteration table (default is True).

    Returns:
    phi_next : float
        Converged phi value.
    """
    phi_current = phi_0

    if show_table:
        print(f"{'Iteration':<10}{'Phi':<15}{'f(Phi)':<20}{'f(Phi)':<20}{'Phi_next':<15}{'Error':<15}")
        print("-" * 95)

    for iteration in range(max_iter):
        f_phi_current = np.sum([(comp_x[i] * (1 - K[i])) / (1 + phi_current * (K[i] - 1)) for i in range(num_components)])
        f_phi_next = np.sum([(comp_x[i] * (1 - K[i])**2) / (1 + phi_current * (K[i] - 1))**2 for i in range(num_components)])

        phi_next = phi_current - f_phi_current / f_phi_next
        error = abs(phi_next - phi_current)

        if show_table:
            print(f"{iteration:<10}{phi_current:<15.6f}{f_phi_current:<20.6f}{f_phi_next:<20.6f}{phi_next:<15.6f}{error:<15.6f}")

        if error < tol:
            return phi_next  # Converged solution

        phi_current = phi_next

    raise ValueError("Newton-Raphson did not converge within the maximum number of iterations")


In [4]:
Newton_Raphson(phi_0, num_components, comp_x, K) #GT3 Problem 1 Part A Work

Iteration Phi            f(Phi)              f(Phi)              Phi_next       Error          
-----------------------------------------------------------------------------------------------
0         0.300000       4.355114            80.493530           0.245895       0.054105       
1         0.245895       -0.170623           87.145249           0.247853       0.001958       
2         0.247853       -0.000273           86.866982           0.247856       0.000003       
3         0.247856       -0.000000           86.866538           0.247856       0.000000       


0.24785591766723838

In [5]:
# GT3 Problem 1 Part A Solution
phi = Newton_Raphson(phi_0, num_components, comp_x, K, show_table=False)
V_F = phi * np.sum(comp_x)
L_F= (1-phi)*np.sum(comp_x)
print(f"Vapor flow rate: {V_F}")
print(f"Liquid flow rate: {L_F}")

Vapor flow rate: 24.78559176672384
Liquid flow rate: 75.21440823327616


In [6]:
#GT3 Problem 1 Part B Solution
# Calculate x and y for each component
x = comp_x / (1 + phi * (K - 1))/100
y = K * x

print(f"Calculated x values: {x}") #In order of component
print(f"Calculated y values: {y}") #In order of component

Calculated x values: [0.19413919 0.80586081]
Calculated y values: [0.62124542 0.37875458]


In [7]:
import numpy as np

def straight_line_error_method(T1, error1, T2, error2):
    """
    Calculate a new temperature using the straight-line error method.

    Parameters:
    T1 : float
        Temperature from the first trial.
    error1 : float
        Error from the first trial.
    T2 : float
        Temperature from the second trial.
    error2 : float
        Error from the second trial.

    Returns:
    new_T : float
        New temperature calculated by linear interpolation.
    """
    # Calculate the slope of the error function
    slope = (error2 - error1) / (T2 - T1)

    # Use linear interpolation to calculate the new temperature
    new_T = T2 - error2 / slope
    return new_T

def bubble_point_temperature(pressure, comp_x, num_components):
    """
    Calculate the bubble point temperature using user input and the straight-line error method.

    Parameters:
    pressure : float
        Pressure at which the bubble point temperature is to be determined (in mmHg).
    comp_x : list or numpy array
        Mole fractions of the components in the mixture.
    num_components : int
        Number of components in the mixture.

    Returns:
    None
    """
    # Prepare to store results for the table
    results = []

    # First trial: Initial guess
    T_guess_1 = float(input("Enter the initial guess temperature (in °C) for trial 1: "))
    K_values_1 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_1}°C: ")) for i in range(num_components)]

    # Calculate the error for the first trial
    error_1 = 1 - np.sum([K_values_1[i] * comp_x[i] for i in range(num_components)])
    results.append((T_guess_1, K_values_1, error_1))

    print(f"Error for Trial 1 (T={T_guess_1}°F): {error_1:.5f}")

    # Second trial: Adjust temperature and K values
    T_guess_2 = float(input("Enter the next temperature guess (in °F) for trial 2: "))
    K_values_2 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_2}°F: ")) for i in range(num_components)]

    # Calculate the error for the second trial
    error_2 = 1 - np.sum([K_values_2[i] * comp_x[i] for i in range(num_components)])
    results.append((T_guess_2, K_values_2, error_2))

    print(f"Error for Trial 2 (T={T_guess_2}°F): {error_2:.5f}")

    # Calculate the new temperature using the straight-line error method
    new_T = straight_line_error_method(T_guess_1, error_1, T_guess_2, error_2)

    # Calculate the error for the new temperature
    K_values_new = [float(input(f"Enter the K value for component {i+1} at T={new_T}°F: ")) for i in range(num_components)]
    error_new = 1 - np.sum([K_values_new[i] * comp_x[i] for i in range(num_components)])
    results.append((new_T, K_values_new, error_new))

    print(f"New temperature after straight-line method: T={new_T:.2f}°F with error {error_new:.5f}")

    # Print the results in a table format
    print(f"\n{'Trial Temp (°F)':<20}{'K values':<40}{'Error':<20}")
    print("="*80)
    for T_guess, K_values, error in results:
        print(f"{T_guess:<20}{' '.join([f'{k:.2f}' for k in K_values]):<40}{error:<20.5f}")




In [9]:
#GT3 Problem 1 Part C Solution
pressure = 100  # Atmospheric pressure in psia
bubble_point_temperature(pressure, comp_x, num_components)

Enter the initial guess temperature (in °C) for trial 1: 140


KeyboardInterrupt: Interrupted by user

In [ ]:
def dew_point_temperature(pressure, comp_x, num_components):
    """
    Calculate the dew point temperature using user input and the straight-line error method.

    Parameters:
    pressure : float
        Pressure at which the dew point temperature is to be determined (in mmHg).
    comp_x : list or numpy array
        Mole fractions of the components in the mixture (liquid phase).
    num_components : int
        Number of components in the mixture.

    Returns:
    None
    """
    # Prepare to store results for the table
    results = []

    # First trial: Initial guess
    T_guess_1 = float(input("Enter the initial guess temperature (in °F) for trial 1: "))
    K_values_1 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_1}°F: ")) for i in range(num_components)]

    # Calculate the error for the first trial
    error_1 = 1 - np.sum([comp_x[i] / K_values_1[i] for i in range(num_components)])
    results.append((T_guess_1, K_values_1, error_1))

    print(f"Error for Trial 1 (T={T_guess_1}°F): {error_1:.5f}")

    # Second trial: Adjust temperature and K values
    T_guess_2 = float(input("Enter the next temperature guess (in °F) for trial 2: "))
    K_values_2 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_2}°F: ")) for i in range(num_components)]

    # Calculate the error for the second trial
    error_2 = 1 - np.sum([comp_x[i] / K_values_2[i] for i in range(num_components)])
    results.append((T_guess_2, K_values_2, error_2))

    print(f"Error for Trial 2 (T={T_guess_2}°F): {error_2:.5f}")

    # Calculate the new temperature using the straight-line error method
    new_T = straight_line_error_method(T_guess_1, error_1, T_guess_2, error_2)

    # Calculate the error for the new temperature
    K_values_new = [float(input(f"Enter the K value for component {i+1} at T={new_T}°F: ")) for i in range(num_components)]
    error_new = 1 - np.sum([comp_x[i] / K_values_new[i] for i in range(num_components)])
    results.append((new_T, K_values_new, error_new))

    print(f"New temperature after straight-line method: T={new_T:.2f}°F with error {error_new:.5f}")

    # Print the results in a table format
    print(f"\n{'Trial Temp (°F)':<20}{'K values':<40}{'Error':<20}")
    print("="*80)
    for T_guess, K_values, error in results:
        print(f"{T_guess:<20}{' '.join([f'{k:.2f}' for k in K_values]):<40}{error:<20.5f}")



In [ ]:
#Problem 1 Part D Solution
dew_point_temperature(pressure, comp_x, num_components)

In [ ]:
#Problem 2
phi_0 = float(input("Input an initial guess for phi: "))
num_components = int(input("Input the number of streams feeding the flash drum: "))
comp_x = np.array(list(map(float, input("Input a list of molar flow rates for each stream (comma-separated): ").split(','))))
K = np.array(list(map(float, input("Input the K (equilibrium constant) values for each stream in order (comma-separated): ").split(','))))

In [ ]:
H=0.31767*5597
M=0.58942*5597
B=0.07147*5597
T=0.02144*5597
print(H, M, B, T)

In [ ]:
#GT3 Problem 2 Part A & B Work
phi= Newton_Raphson(phi_0, num_components, comp_x, K)
V_F = phi * np.sum(comp_x)
L_F= (1-phi)*np.sum(comp_x)
print(f"Vapor flow rate: {V_F}")
print(f"Liquid flow rate: {L_F}")

In [ ]:
#GT3 Problem 2 Part C Solution
# Calculate x and y for each component
x = comp_x / (1 + phi * (K - 1))/100
y = K * x

print(f"Calculated x values: {x}") #In order of component
print(f"Calculated y values: {y}") #In order of component

**Problem 3 - Distillation**